#### 1. Import pandas library

In [2]:
import pandas as pd

#### 2. Import pymysql and sqlalchemy as you have learnt in the lesson of importing/exporting data 


In [3]:
import pymysql
# from sqlalchemy import create_engine
import sqlalchemy as db
import getpass

#### 3. Create a mysql engine to set the connection to the server. Check the connection details in [this link](https://relational.fit.cvut.cz/search?tableCount%5B%5D=0-10&tableCount%5B%5D=10-30&dataType%5B%5D=Numeric&databaseSize%5B%5D=KB&databaseSize%5B%5D=MB)

In [4]:
# https://relational.fit.cvut.cz/dataset/Stats
# https://relational.fit.cvut.cz/assets/img/datasets-generated/stats.svg

password = getpass.getpass("Insert your mysql root password: ")
engine = db.create_engine('mysql+pymysql://guest:{}@relational.fit.cvut.cz/stats'.format(password))
print("Connected to server!")

Insert your mysql root password: ········
Connected to server!


#### 4. Import the users table 

In [5]:
UsersDf = pd.read_sql_query('SELECT * FROM users', engine)
display(UsersDf.head())

,Id,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


#### 5. Rename Id column to userId

In [6]:
UsersDf = UsersDf.rename(columns={'Id': 'userId'})
display(UsersDf.head())

,userId,Reputation,CreationDate,DisplayName,LastAccessDate,WebsiteUrl,Location,AboutMe,Views,UpVotes,DownVotes,AccountId,Age,ProfileImageUrl
0,-1,1,2010-07-19 06:55:26,Community,2010-07-19 06:55:26,http://meta.stackexchange.com/,on the server farm,"<p>Hi, I'm not really a person.</p>\n\n<p>I'm ...",0,5007,1920,-1,NaN,None
1,2,101,2010-07-19 14:01:36,Geoff Dalgas,2013-11-12 22:07:23,http://stackoverflow.com,"Corvallis, OR",<p>Developer on the StackOverflow team. Find ...,25,3,0,2,37.0,None
2,3,101,2010-07-19 15:34:50,Jarrod Dixon,2014-08-08 06:42:58,http://stackoverflow.com,"New York, NY","<p><a href=""http://blog.stackoverflow.com/2009...",22,19,0,3,35.0,None
3,4,101,2010-07-19 19:03:27,Emmett,2014-01-02 09:31:02,http://minesweeperonline.com,"San Francisco, CA",<p>currently at a startup in SF</p>\n\n<p>form...,11,0,0,1998,28.0,http://i.stack.imgur.com/d1oHX.jpg
4,5,6792,2010-07-19 19:03:57,Shane,2014-08-13 00:23:47,http://www.statalgo.com,"New York, NY",<p>Quantitative researcher focusing on statist...,1145,662,5,54503,35.0,None


#### 6. Import the posts table. 

In [7]:
PostsDf = pd.read_sql_query('SELECT * FROM posts', engine)
display(PostsDf.head(5))

,Id,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,OwnerUserId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaT,NaT,NaN,NaT,None,None
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaT,NaN,NaT,None,None
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaT,None,None
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaT,NaT,NaN,NaT,None,None
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,None,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaT,None,None


#### 7. Rename Id column to postId and OwnerUserId to userId

In [8]:
PostsDf = PostsDf.rename(columns={'Id': 'postId', 'OwnerUserId': 'userId'})
display(PostsDf.head(5))

,postId,PostTypeId,AcceptedAnswerId,CreaionDate,Score,ViewCount,Body,userId,LasActivityDate,Title,...,AnswerCount,CommentCount,FavoriteCount,LastEditorUserId,LastEditDate,CommunityOwnedDate,ParentId,ClosedDate,OwnerDisplayName,LastEditorDisplayName
0,1,1,15.0,2010-07-19 19:12:12,23,1278.0,<p>How should I elicit prior distributions fro...,8.0,2010-09-15 21:08:26,Eliciting priors from experts,...,5.0,1,14.0,NaN,NaT,NaT,NaN,NaT,None,None
1,2,1,59.0,2010-07-19 19:12:57,22,8198.0,<p>In many different statistical methods there...,24.0,2012-11-12 09:21:54,What is normality?,...,7.0,1,8.0,88.0,2010-08-07 17:56:44,NaT,NaN,NaT,None,None
2,3,1,5.0,2010-07-19 19:13:28,54,3613.0,<p>What are some valuable Statistical Analysis...,18.0,2013-05-27 14:48:36,What are some valuable Statistical Analysis op...,...,19.0,4,36.0,183.0,2011-02-12 05:50:03,2010-07-19 19:13:28,NaN,NaT,None,None
3,4,1,135.0,2010-07-19 19:13:31,13,5224.0,<p>I have two groups of data. Each with a dif...,23.0,2010-09-08 03:00:19,Assessing the significance of differences in d...,...,5.0,2,2.0,NaN,NaT,NaT,NaN,NaT,None,None
4,5,2,NaN,2010-07-19 19:14:43,81,NaN,"<p>The R-project</p>\n\n<p><a href=""http://www...",23.0,2010-07-19 19:21:15,None,...,NaN,3,NaN,23.0,2010-07-19 19:21:15,2010-07-19 19:14:43,3.0,NaT,None,None


#### 8. Define new dataframes for users and posts with the following selected columns:
    **users columns**: userId, Reputation,Views,UpVotes,DownVotes
    **posts columns**: postId, Score,userID,ViewCount,CommentCount

In [9]:
UsersDf_new = UsersDf[['userId','Reputation','Views','UpVotes','DownVotes']]
PostsDf_new = PostsDf[['postId','Score','userId','ViewCount','CommentCount']]
display(UsersDf_new.head(5))
display(PostsDf_new.head(5))

,userId,Reputation,Views,UpVotes,DownVotes
0,-1,1,0,5007,1920
1,2,101,25,3,0
2,3,101,22,19,0
3,4,101,11,0,0
4,5,6792,1145,662,5


,postId,Score,userId,ViewCount,CommentCount
0,1,23,8.0,1278.0,1
1,2,22,24.0,8198.0,1
2,3,54,18.0,3613.0,4
3,4,13,23.0,5224.0,2
4,5,81,23.0,NaN,3


#### 8. Merge both dataframes, users and posts. 
You will need to make a [merge](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.DataFrame.merge.html) of posts and users dataframes.

In [24]:
MergedDf = pd.merge(UsersDf_new, PostsDf_new, on='userId')
display(MergedDf.head(5))

,userId,Reputation,Views,UpVotes,DownVotes,postId,Score,ViewCount,CommentCount
0,-1,1,0,5007,1920,2175,0,NaN,0
1,-1,1,0,5007,1920,8576,0,NaN,0
2,-1,1,0,5007,1920,8578,0,NaN,0
3,-1,1,0,5007,1920,8981,0,NaN,0
4,-1,1,0,5007,1920,8982,0,NaN,0


#### 9. How many missing values do you have in your merged dataframe? On which columns?

In [25]:
print(MergedDf.isnull().sum())

userId              0
Reputation          0
Views               0
UpVotes             0
DownVotes           0
postId              0
Score               0
ViewCount       48396
CommentCount        0
dtype: int64


#### 10. You will need to make something with missing values.  Will you clean or filling them? Explain. 
**Remember** to check the results of your code before passing to the next step

In [26]:
# Thera are a lot of missing values in the column ViewCount:
DF0 = MergedDf[MergedDf['ViewCount'].isnull()]
print("Missing values in ViewCount:", DF0.shape[0],"rows out of",MergedDf.shape[0])

# Furthermore, there are some rows with a ViewCount = NaN and a CommentCount > 0, what makes no sense:
DF1 = MergedDf[MergedDf['ViewCount'].isnull() & MergedDf['CommentCount'] > 0]
print("ViewCount = NaN and CommentCount > 0:",DF1.shape[0],"rows out of",DF0.shape[0])

# The variable ViewCount does not seem very reliable...

Missing values in ViewCount: 48396 rows out of 90584
ViewCount = NaN and CommentCount > 0: 15338 rows out of 48396


#### 11. Adjust the data types in order to avoid future issues. Which ones should be changed? 

In [27]:
# print(MergedDf.dtypes)

# ViewCount should be an integer. In order to change the data type...

MergedDf[['ViewCount']] = MergedDf[['ViewCount']].fillna(0)
MergedDf['ViewCount'] = MergedDf['ViewCount'].astype(int)

# print(MergedDf.dtypes)

#### Bonus: Identify extreme values in your merged dataframe as you have learned in class, create a dataframe called outliers with the same columns as our data set and calculate the bounds. The values of the outliers dataframe will be the values of the merged_df that fall outside that bounds. You will need to save your outliers dataframe to a csv file on your-code folder.

In [30]:
stats = MergedDf.describe().T
stats['IQR'] = stats['75%'] - stats['25%']
stats

,count,mean,std,min,25%,50%,75%,max,IQR
userId,90584.0,16546.764727,15273.367108,-1.0,3437.00,11032.0,27700.00,55746.0,24263.00
Reputation,90584.0,6282.395412,15102.268670,1.0,60.00,396.0,4460.00,87393.0,4400.00
Views,90584.0,1034.245176,2880.074012,0.0,5.00,45.0,514.25,20932.0,509.25
UpVotes,90584.0,734.315718,2050.869327,0.0,1.00,22.0,283.00,11442.0,282.00
DownVotes,90584.0,33.273249,134.936435,0.0,0.00,0.0,8.00,1920.0,8.00
postId,90584.0,56539.080522,33840.307529,1.0,26051.75,57225.5,86145.25,115378.0,60093.50
Score,90584.0,2.780767,4.948922,-19.0,1.00,2.0,3.00,192.0,2.00
ViewCount,90584.0,259.253400,1632.261405,0.0,0.00,0.0,111.00,175495.0,111.00
CommentCount,90584.0,1.894650,2.638704,0.0,0.00,1.0,3.00,45.0,3.00


In [31]:

outliers = pd.DataFrame(columns=MergedDf.columns)

for col in stats.index:
    iqr = stats.at[col,'IQR']
    cutoff = iqr * 1.5
    lower = stats.at[col,'25%'] - cutoff
    upper = stats.at[col,'75%'] + cutoff
    results = MergedDf[(MergedDf[col] < lower) | 
                   (MergedDf[col] > upper)].copy()
    results['Outlier'] = col
    outliers = outliers.append(results)


C:\ProgramData\Anaconda3\lib\site-packages\pandas\core\frame.py:6692: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  sort=sort)


In [32]:
display(outliers)

,CommentCount,DownVotes,Outlier,Reputation,Score,UpVotes,ViewCount,Views,postId,userId
1155,0,126,Reputation,14082,25,4235,0,3320,74,88
1156,0,126,Reputation,14082,5,4235,0,3320,94,88
1157,1,126,Reputation,14082,7,4235,0,3320,99,88
1158,3,126,Reputation,14082,6,4235,0,3320,119,88
1159,0,126,Reputation,14082,7,4235,0,3320,140,88
1160,2,126,Reputation,14082,5,4235,0,3320,143,88
1161,1,126,Reputation,14082,8,4235,0,3320,255,88
1162,0,126,Reputation,14082,14,4235,0,3320,265,88
1163,0,126,Reputation,14082,5,4235,0,3320,275,88
1164,1,126,Reputation,14082,2,4235,0,3320,309,88
